In [1]:

import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

In [3]:
#config_entity.py

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path #going to download VGG16 model
    updated_base_model_path: Path #Fine tuned VGG16 model
    params_image_size: list  #We are using [224,224,3] where 224 repesents length and width of the image whereas, 3 represents 3 channels(RGB) since here we are using colored images.
    params_learning_rate: float #self understood
    params_include_top: bool #To perform fine tuning where we used to freeze upto ANN layer and train the ANN layer in CNN. If set to True means include ANN layer in the CNN base of repective vision model like VGG16 else if False means do not include that ANN layer
    params_weights: str
    params_classes: int

In [4]:
#Constant/__init__.py and utils.py

from CNNClassifier.constants import *
from CNNClassifier.utils.utils import read_yaml, create_directory

In [5]:
#configuration.py

class ConfigurationManager:
    def __init__(
        self, config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directory([self.config.artifacts_root])
        

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directory([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [7]:
#src/CNNClassifier/components/stage_02_prepare_base_model.py

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        #below we are basically dowloading the raw or base VGG16 model from tensor flow by passing
        #different attributes which we already discussed.
        self.model = tf.keras.applications.vgg16.VGG16(   
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        #below we are saving the model. Here save_model is another function in the same class.
        #Alternatively, we can also place this save_model func^ inside the utils folder.
        self.save_model(path=self.config.base_model_path, model=self.model)


    @staticmethod
    def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):

        #If freeze_all is set to True the we are going to freeze all the layers of the VGG16 model which
        #means we are not going to update the pre trained weights that why writing
        #model.trainable = False.
        if freeze_all:
            for layer in model.layers:
                model.trainable = False

        #Else we are going to train the freeze the layers upto freeze_till by witing model.trainable = False
        # and beyond that we are going to train the layers until end. Say we for model with 10 layers we are
        #going to freeze until 8 layers and we are going to train the last 2 layers. This is how
        #we are going to fine tune the VGG16 base model on top of our ingested data. These all are the
        #hyperparameters.
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        #Below is the functional architecture that was explained in .ipynb file during lecture for
        #flattening and this flattened layer in then passed to the fully connected ANN layer.
        flatten_in = tf.keras.layers.Flatten()(model.output)
        #Again we are calling the functional architecture. need to recall basics
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self.prepare_full_model(  #this is another function in the same class
            model=self.model,  #VGG16
            classes=self.config.params_classes, #2
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate #0.01
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [8]:
#src/CNNClassifier/pipline/stage_02_prepare_base_model.py

try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

##Observe below we are getting the logs of artifact getting created
##which is coming since in utils we while defining the respective functionality
#we embedded logs

[2023-10-01 16:02:50,273: INFO: utils]: create directory at : artifacts
[2023-10-01 16:02:50,275: INFO: utils]: create directory at : artifacts/prepare_base_model


[2023-10-01 16:02:51,357: WARNING: saving_utils]: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


c:\Complete_Content\All_Project\fdsdcnnclassifier\env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     